### 方法
```
while 調べるべき特徴量がある限り :
    for 調べる特徴量 in 調べる特徴量の数 :
        ある特徴量ベクトルxiと元のy(y^)の内積を求める(t^i)
        for 100程度 :
            y^を、{t0 = t^0かつt1 = t^1...tj = t^j}という条件の元サンプリング（nealを使う）しyを生成する。
            ある特徴量ベクトルxiとyの内積を求める(ti)
        複数のtiを求めることによって得られた分布に対するt^iの位置から、p値を求める。
    p値の一番小さいXsを知る。
    条件{}にts = t^sを足す（更新する）
    調べる特徴量からXsを除外（更新する）
```

#### 条件について
- t0 = t^0 : 1の総和が同じ
- tj = t^j : xの特徴量jつ目とyの内積の値が同じ


### yのサンプリング
- 条件に合うまでサンプルを作り続ける

In [1]:
#!pip3 install dwave-neal

In [2]:
import pandas as pd 
import numpy as np
from neal import SimulatedAnnealingSampler
from pyqubo import Array, Constraint, Placeholder, solve_qubo
import random

In [3]:
X_mushroom = pd.read_csv("../input/SNP_df1010.csv", sep=',', index_col=0)
y_mushroom = pd.read_csv('../input/y10.csv', sep=',', index_col=0, dtype=np.float64)['y']

num_samples = X_mushroom.shape[0]
num_cols = X_mushroom.shape[1]
columns = list(X_mushroom.columns)

y_sum = sum(y_mushroom)

In [4]:
num_samples

10

In [5]:
y_sum

6.0

In [6]:
def make_y(minP_cols):
    while True:
        y = [[0, 1][random.randint(0, 1)]for jj in range(num_samples)]
        if sum(y) - y_sum != 0:
            continue
        if len(minP_cols) == 0:
            return pd.Series(y)
        else:
            for i in minP_cols:
                if np.dot(X_mushroom[i], y) - np.dot(X_mushroom[i], y_mushroom) != 0:
                    continue
            return pd.Series(y)

In [7]:
minP_cols = []
servey_num_col_list = columns

while len(servey_num_col_list) > 0:    
    p = {}
    for i in servey_num_col_list:
        ori_ti = int(np.dot(X_mushroom[i], y_mushroom)) #特徴量iを表すp
        
        ti_dic = {} #分布の要素となる値を格納する
        for _ in range(100): 
            y = make_y(minP_cols)
            
            if int(np.dot(X_mushroom[i], y)) in ti_dic.keys():
                ti_dic[int(np.dot(X_mushroom[i], y))] += 1
            else:
                ti_dic[int(np.dot(X_mushroom[i], y))] = 1
                
        bunshi = sum([ti_dic[ii] for ii in range(ori_ti, int(list(ti_dic.keys())[-1])+1)])
        p[i] = bunshi/sum(list(ti_dic.values()))
        
    minP_keys = [k for k, v in p.items() if v == min(list(p.values()))]
    minP_cols += minP_keys
    servey_num_col_list = [i for i in servey_num_col_list if i not in minP_keys]

In [8]:
minP_cols

['B', 'F', 'G', 'A', 'C', 'D', 'E', 'H', 'I', 'J']